In [4]:
import pandas as pd
import os

const_dir = 'Data_20210409/'

file_dir = os.listdir(const_dir)


In [5]:
file_dir

['202101.csv', '202102.csv', '202103.csv', 'APP.csv', 'TV', 'WEB.csv']

# Concatenate Main File

In [6]:
_list = ['202103.csv']

dfs = []

for _l in _list:
    dfs.append(pd.read_csv(const_dir + _l))
    
_df = pd.concat(dfs)

C:\Users\Asus ROG\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
app = pd.read_csv(const_dir + 'APP.csv')
web = pd.read_csv(const_dir + 'WEB.csv')

from functools import reduce

df = reduce(lambda left,right: pd.merge(left,right,on=['MONTH_SQN', 'SUB_SQN'], how='inner'), [_df, app, web])

In [8]:
df['GENDER'].replace('UNKNOWN', df['GENDER'].mode()[0], inplace=True)

In [9]:
df['PREDICTED_RACE'].replace('UNKNOWN', df['PREDICTED_RACE'].mode()[0], inplace=True)

In [10]:
df = df[df['MONTH_SQN'] == 202103]

In [11]:
len(df.columns)

175

## A. Numerical Columns groupby mean

In [12]:
const_col = ['SERVICE','MONTH_SQN', 'CUST_SQN']
cat_col = ['PRODUCT_CODE', 'GROUP_PRODUCT_CODE_LV1', 'GROUP_PRODUCT_CODE_LV2', 'CIS_STATUS', 'SUBS_TENURE', 'PORT_IN_FLAG']
object_col = [c for c in df.columns if df[c].dtype == 'object' and c not in const_col] + cat_col

df_num = df.groupby(by=const_col).mean().reset_index().drop(columns=['SUB_SQN']+ cat_col + ['AUSAGE_TOTAL', 'USAGE_TOTAL'])

In [15]:
len(df_num.columns)

145

## B. Special Preprocessing of categorical columns prior to groupby Counter

In [ ]:
for c in cat_col:
    df[c] = df[c].apply(lambda x: str(x))

## C. Categorical Columns groupby Counter

In [ ]:
from collections import Counter
from typing import Dict, Any
import numpy as np

df[object_col] = df[object_col].fillna("")

def _get_df_obj(df: pd.DataFrame) -> pd.DataFrame:
    """  
    To squash columns of data types == 'object' into Dictionary of category and counts 
    for each cell.
    
    Example
    =========
    {'OPP_R15 PRO': 1, 'SAM_TAB S5E 128GB': 1}
    """
    def _get_counter(x: pd.Series) -> Dict[Any, int]:
        tmp = x.split(",")
        if len(tmp) == 1 and tmp[0] == "":
            return np.nan
        
        res = {k: v for k, v in Counter(tmp).items()} 
        
        if len(res) == 1 and list(res.keys())[0] == "":
            return np.nan
        
        return {k: v for k, v in res.items() if k != ""}
    
    df_obj = df.groupby(by=const_col)[object_col].agg(lambda x: ",".join(x))

    for c in df_obj.columns:
        df_obj[c] = df_obj[c].apply(lambda x: _get_counter(x))
    
    return df_obj.reset_index()

df_obj = _get_df_obj(df)

In [ ]:
df_obj.columns

## D. Merge Num and Categorical dataset

In [ ]:
res = df_num.merge(df_obj, how='left', on=const_col)
# res = res[res['MONTH_SQN'] == 202103]

# Check missing data

In [ ]:
_res = pd.DataFrame(res.isna().sum()/len(res) * 100)
_res.columns = ['missing_pct']

In [ ]:
_res = _res[_res['missing_pct'] > 30] 

In [ ]:
_res

# Remove columns which missing data > 30%¶

In [ ]:
def get_missing_data_perc(x: pd.Series) -> float:
    return x.isna().sum()/len(x) * 100

usable_col = [c for c in res.columns if get_missing_data_perc(res[c]) < 30] 

In [ ]:
res_usable = res[usable_col]

# Impute missing data with Mode

In [ ]:
for c in res_usable.columns:
    res_usable[c].fillna(res_usable[c].mode()[0], inplace=True)


In [ ]:
## columns required 1 hot encoding: PLAN_TIER, ACQ_TYPE, CONTRACT_STATUS, SALES_CHANNEL_GROUP, 'ORDER_CPE_BRAND', 'GROUP_PRODUCT_CODE_LV1', 'GROUP_PRODUCT_CODE_LV2'
## columns binary: 'PRODUCT_CODE', 'CIS_STATUS', 'SUBS_TENURE', 'PORT_IN_FLAG', 
## columns to drop: 'CUST_TENURE', 'SERVICE', 'SUBS_STATUS', 'HC_STATUS', 'DISTRICT', 'DWELLING_ROOM_TYPE', 'PAYMENT_METHOD', 
## 'MONTHS_SINCE_CONTRACT_START', 'MONTHS_TO_CONTRACT_END', 'MTHS_WITH_OVERDUE_BILL_6M'
## 'VOL_MEDIA_APP', 'DAYS_ACTV_GAME_APP', 'DAYS_ACTV_SPORT_APP', 'DAYS_ACTV_IM_APP', 'TIMES_USED_SM_APP', 'VOL_SM_APP', 'MTHS_FRM_LAST_HANDSET_CHANGE',
## 'HANDSETS_USED_IN_5YRS', 'CURR_HANDSET_BRAND', 'CURR_HANDSET_MODEL', 'ORDER_CPE_MODEL', 'ORDER_CPE_OS'
## columns to get keys: 'GENDER', 'PREDICTED_RACE', 'NATIONALITY'

# Drop columns not relevant to clustering

In [ ]:
columns_to_drop = ['CUST_TENURE', 'SERVICE', 'SUBS_STATUS', 'HC_STATUS', 'DISTRICT', 
                   'DWELLING_ROOM_TYPE', 'PAYMENT_METHOD', 'MONTHS_SINCE_CONTRACT_START', 
                   'MONTHS_TO_CONTRACT_END', 'MTHS_WITH_OVERDUE_BILL_6M', 'VOL_MEDIA_APP', 
                   'DAYS_ACTV_GAME_APP', 'DAYS_ACTV_SPORT_APP', 'DAYS_ACTV_IM_APP', 
                   'TIMES_USED_SM_APP', 'VOL_SM_APP', 'MTHS_FRM_LAST_HANDSET_CHANGE',
                  'HANDSETS_USED_IN_5YRS', 'CURR_HANDSET_BRAND', 'CURR_HANDSET_MODEL', 
                   'ORDER_CPE_MODEL', 'ORDER_CPE_OS', 'TIMES_USED_IM_APP', 'VOL_IM_APP', 'DAYS_ACTV_SM_APP',
                  'NATIONALITY']

In [ ]:
res_usable.drop(columns=columns_to_drop, inplace=True)

In [ ]:
# SUBS_TENURE => to take mean of keys
# 'PRODUCT_CODE', 'CIS_STATUS', 'SUBS_TENURE', 'PORT_IN_FLAG', 'GROUP_PRODUCT_CODE_LV1', 'GROUP_PRODUCT_CODE_LV2' 
# => to take '1' if value >= '0' else 0
# 'PLAN_TIER', 'ACQ_TYPE', 'CONTRACT_STATUS', 'SALES_CHANNEL_GROUP', 'ORDER_CPE_BRAND', 'GENDER', 'PREDICTED_RACE', 
# 'NATIONALITY' => to take values of key if key exist elses Na; return Na if Na

# Further Preprocessing for Columns with Counter

In [ ]:
from statistics import mean
import math

res_usable['SUBS_TENURE'] = res_usable['SUBS_TENURE'].apply(lambda x: mean([int(k) for k in list(x.keys())]))



In [ ]:
binary_columns = ['CIS_STATUS', 'PORT_IN_FLAG', 'GROUP_PRODUCT_CODE_LV1', 'GROUP_PRODUCT_CODE_LV2']

def _get_binary_value(x) -> int:
    if len(x) == 1:
        return int(list(x.keys())[0])
    return 1 if x['1'] >= x['0'] else 0

for b in binary_columns:
    res_usable[b] = res_usable[b].apply(lambda x: _get_binary_value(x))





In [ ]:
one_hot_encoded_columns = ['PRODUCT_CODE', 'PLAN_TIER', 'ACQ_TYPE', 'CONTRACT_STATUS', 
                           'SALES_CHANNEL_GROUP', 'ORDER_CPE_BRAND', 
                           'GENDER', 'PREDICTED_RACE', 'AGE_GROUP']

def _get_value(x) -> int:
    if not isinstance(x, dict):
        return np.nan
    if c in x.keys():
        return 1 #x[c]
    return np.nan

for o in one_hot_encoded_columns:
    col = df[o].unique()
    if o == 'PRODUCT_CODE':
        for c in col:
            res_usable[f'PRODUCT_CODE_{c}'] = res_usable[o].apply(lambda x: _get_value(x))
    elif o == 'GENDER':
        for c in col:
            res_usable[f'GENDER_{c}'] = res_usable[o].apply(lambda x: _get_value(x))
    else:
        for c in col:
            res_usable[c] = res_usable[o].apply(lambda x: _get_value(x))

In [ ]:
# pd.set_option('display.max_rows', 300)
# res_usable.T

In [ ]:
## Further drop the following columns
## 'MONTH_SQN', 'CUST_SQN', 'CUST_LVL_FBB_ARPU_6M', 'CUST_LVL_TV_ARPU_6M', 'MTHS_BTW_HANDSET_CHANGE', 'GENDER',
## 'PREDICTED_RACE', 'AGE_GROUP', 'PLAN_TIER', 'ACQ_TYPE', 'CONTRACT_STATUS', 'SALES_CHANNEL_GROUP', 'ORDER_CPE_BRAND',
## 'PRODUCT_CODE', 'Unknown', ''

## Columns requires normalization
## 'SH_CUST_RATING', 'CUST_LVL_MOB_ARPU_6M', 'TENURE_MOBILE', 'CUST_LVL_MOB_CNT', 'CONTRACT_DURATION', 
## 'LIFETIME_CONTRACTS_CNT', 'AUSAGE_SOCIALNET',
## 'AUSAGE_VIDEO', 'AUSAGE_COMMUNICATIONS', 'AUSAGE_NETFLIX',
## 'AUSAGE_ECOMMERCE', 'AUSAGE_GAMES', 'AUSAGE_MUSIC', 'AUSAGE_MAIL',
## 'AUSAGE_LIFESTYLE', 'AUSAGE_NEWS', 'AUSAGE_TRANSPORTATION'
## 'AUSAGE_NEWS', 'AUSAGE_TRANSPORTATION'
## USAGE_SOCIALNET', 'USAGE_STREAMINGVIDEO', 'USAGE_GAMES', 'USAGE_ENTERTAINMENT',
## 'USAGE_MUSIC', 'USAGE_TRANSPORTATION', 'USAGE_SHOPPING',
## 'USAGE_ECOMMERCE', 'USAGE_SPORTS', 'USAGE_NEWS', 'USAGE_TRAVEL'

# Normalization, Fill One-hot encoded columns with 0

In [ ]:
res_usable.drop(columns=['MONTH_SQN', 'CUST_SQN', 'CUST_LVL_FBB_ARPU_6M', 'CUST_LVL_TV_ARPU_6M', 
                         'MTHS_BTW_HANDSET_CHANGE', 'GENDER', 'PREDICTED_RACE', 'AGE_GROUP', 'PLAN_TIER', 
                         'ACQ_TYPE', 'CONTRACT_STATUS', 'SALES_CHANNEL_GROUP', 'ORDER_CPE_BRAND',
                         'PRODUCT_CODE', 'Unknown', '',], inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler

normalize_col = ['SH_CUST_RATING', 'CUST_LVL_MOB_ARPU_6M', 'TENURE_MOBILE', 'CUST_LVL_MOB_CNT', 'CONTRACT_DURATION', 'SUBS_TENURE',
                'LIFETIME_CONTRACTS_CNT', 'AUSAGE_SOCIALNET', 'AUSAGE_VIDEO', 'AUSAGE_COMMUNICATIONS', 'AUSAGE_NETFLIX',
                'AUSAGE_ECOMMERCE', 'AUSAGE_GAMES', 'AUSAGE_MUSIC', 'AUSAGE_MAIL','AUSAGE_LIFESTYLE', 'AUSAGE_NEWS', 'AUSAGE_TRANSPORTATION',
                'AUSAGE_NEWS', 'AUSAGE_TRANSPORTATION', 'USAGE_SOCIALNET', 'USAGE_STREAMINGVIDEO', 'USAGE_GAMES', 'USAGE_ENTERTAINMENT',
                'USAGE_MUSIC', 'USAGE_TRANSPORTATION', 'USAGE_SHOPPING', 'USAGE_ECOMMERCE', 'USAGE_SPORTS', 'USAGE_NEWS', 'USAGE_TRAVEL']

scaler = StandardScaler()
scaler.fit(res_usable[normalize_col])
scaled_target = res_usable[normalize_col].copy()

res_usable[normalize_col]= scaler.transform(scaled_target)

In [ ]:
res_usable.fillna(0, inplace=True)